In [ ]:
## This is to try to understand (and in the future automatically calculate and visualize) how the KFSI are calculated based on individual IDS. 

In [81]:
import pandas as pd
import numpy as np

import pylab as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline

import altair as alt
def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % tuple(np.array(255*rgb,dtype=int))



# Read raw data and define the indicators

In [11]:
k1_ind = [89,157,158,352,353,360]

fsi = pd.read_excel("Country-level/Sources/FSI_MySQL_dump/20191216_Matrix_Full.xlsx")
fsi.head()

,Period,Jurisdiction,Info_ID,DataType,Question,Lkup_Code,Lkup_Text,Nmbr,Text
0,18,AD,2,Lkup,Jurisdiction Status,2,Dependent Territory,NaN,NaN
1,20,AD,2,Lkup,Jurisdiction Status,2,Dependent Territory,NaN,NaN
2,18,AD,4,Num,Population,-4,-/-,85660.000,NaN
3,18,AD,6,Num,HDI Rank: Human Development Index Rank,-4,-/-,34.000,NaN
4,18,AD,7,Num,HDI Value: Human Development Index Value,-4,-/-,0.845,NaN


In [12]:
def print_questions(data):
    """
    Print the questions associated to the IDs, return a dictionary with the matching
    """
    d_questions = dict()
    
    #Print questions
    print("Questions")
    for i,row in data[["Info_ID","Question"]].drop_duplicates(subset=["Info_ID"]).iterrows():
        print("{}: {}".format(row["Info_ID"],row["Question"]))
        d_questions[row["Info_ID"]] = row["Question"]
        
    return d_questions

# KFSI 1: Banking Secrecy
- All questions here are categorical

In [13]:
k1 = fsi.loc[fsi["Info_ID"].isin(k1_ind)]

d_questions_k1 = print_questions(k1)
    
k1.head()

Questions
89: *Are banks and/or other covered entities required to report large transactions in currency or other monetary instruments to designated authorities?
157: *Does the domestic administration have sufficient powers to obtain and provide banking information on request?
158: *Are there no undue notification and appeal rights against bank information exchange on request?
352: *To what extent are banks subject to stringent customer due diligence regulations (Old FATF-recommendation 5 / new FATF-recommendation 10)?
353: *To what extent are banks required to maintain data records of its customers and transactions sufficient for law enforcement (old FATF-recommendation 10 / new FATF recommendation 11)?
360: *Are there criminal sanctions, custodial sentencing or any other statutory sanctions for breaches of banking secrecy?


,Period,Jurisdiction,Info_ID,DataType,Question,Lkup_Code,Lkup_Text,Nmbr,Text
51,18,AD,89,YN,*Are banks and/or other covered entities requi...,0,No,NaN,NaN
52,20,AD,89,YN,*Are banks and/or other covered entities requi...,0,No,NaN,NaN
109,18,AD,157,Lkup,*Does the domestic administration have suffici...,3,"Yes, but major problems.",NaN,NaN
110,20,AD,157,Lkup,*Does the domestic administration have suffici...,3,"Yes, but major problems.",NaN,NaN
111,18,AD,158,Lkup,*Are there no undue notification and appeal ri...,3,"Yes, but major problems.",NaN,NaN


In [14]:
#Questions are categorical, so we can use the lkup_text. Pivot_table doesn't like categorical variables, so we concatenate the values (there should be only one, but just in case)
k1_pivot_or = pd.pivot_table(k1,values="Lkup_Code",index=["Period","Jurisdiction"],columns="Info_ID")

k1_pivot = k1_pivot_or.copy()
#Transformatations
k1_pivot[360] = (k1_pivot[360]<=0).astype(float) #Unknown or prison
k1_pivot[352] = k1_pivot[352]/3 #extend to regulations
k1_pivot[353] = k1_pivot[353]/3 #extent to maintain recors
k1_pivot[89] = (k1_pivot[89]<=0).astype(float) #Unknown or banks not required to report
k1_pivot[157] = (k1_pivot[157]!=1).astype(float) #sufficient powers
k1_pivot[158] = (k1_pivot[158]!=1).astype(float) #appeals agains info exchange

weights = np.array([0.2,0.1,0.1,0.2,0.2,0.2])

k1_weighted = np.sum(k1_pivot*weights,axis=1)*100
k1_weighted = k1_weighted.reset_index()
k1_weighted.columns = ["Period","Jurisdiction","KFSI_1"]
k1_weighted.head()

,Period,Jurisdiction,KFSI_1
0,18,AD,86.666667
1,18,AE,46.666667
2,18,AG,93.333333
3,18,AI,70.000000
4,18,AT,56.666667


In [34]:
#Define graphs (move up later)
sns.set_palette(sns.diverging_palette(230, 15, n=7))

In [16]:
secrecy_thr = [0,40,50,60,70,80,90,100]
k1_weighted["class"] = pd.cut(k1_weighted["KFSI_1"],bins=secrecy_thr,include_lowest=True,labels=["0-40","41-50","51-60","61-70","71-80","81-90","91-100"])
k1_weighted.head()

,Period,Jurisdiction,KFSI_1,class
0,18,AD,86.666667,81-90
1,18,AE,46.666667,41-50
2,18,AG,93.333333,91-100
3,18,AI,70.000000,61-70
4,18,AT,56.666667,51-60


In [116]:
color_scale = alt.Scale(
    domain=[
        "0-40",
        "41-50",
        "51-60",
        "61-70",
        "71-80",
        "81-90",
        "91-100"
    ],
    range=[rgb_to_hex(x[:3]) for x in sns.diverging_palette(230, 15, n=7)]
)


chart = alt.Chart(k1_weighted).mark_bar().encode(
    y='Period:O',
    x='count(Period):Q',
    color=alt.Color(
        'class:N',
        legend=alt.Legend( title='Secrecy'),
        scale=color_scale,
    )
)
chart

alt.Chart(...)

In [103]:
d_coded = {'No':'#d04739',
  'Please enter':'#e69c95',
  'Unknown':'#e69c95',
  'Yes':'#3f7e9b',
 
  "Yes, there are prison terms for disclosing client's banking data to any third party (and possibly fines).":'#d04739',
  "Yes, there are fines for disclosing client's banking data to any third party, but no prison terms.":'#e69c95',
 "No, there are no statutory sanctions for disclosing client's banking data to any third party.":'#3f7e9b',
 
 'No, access is not possible, or only exceptionally.':'#d04739',
 'Yes, but major problems.':'#e69c95',
  'Yes, but some problems.':'#f2f2f2',
 'Yes without qualifications.':'#9abccb',
 
 'Fully.':'#3f7e9b',
 'Largely.':'#9abccb',
 'Partially.': '#e69c95',
 'Not at all.': '#d04739'}
      
       


In [111]:

for i,data in k1.groupby("Question"):
    data = data.dropna(subset=["Jurisdiction"])
    domain = list(data["Lkup_Text"].unique())
    color_scale = alt.Scale(
    domain=domain,
    range=[d_coded[_] for _ in domain]
)

    
    
    print(i)
    chart = alt.Chart(data).mark_bar().encode(
        x="count(Lkup_Text):O",
        color=alt.Color(
            'Lkup_Text:N',
            scale=color_scale,
            ),
        y="Period:O"
    )

    display(chart)


*Are banks and/or other covered entities required to report large transactions in currency or other monetary instruments to designated authorities?


alt.Chart(...)

*Are there criminal sanctions, custodial sentencing or any other statutory sanctions for breaches of banking secrecy?


alt.Chart(...)

*Are there no undue notification and appeal rights against bank information exchange on request?


...

alt.Chart(...)

*To what extent are banks required to maintain data records of its customers and transactions sufficient for law enforcement (old FATF-recommendation 10 / new FATF recommendation 11)?


alt.Chart(...)

*To what extent are banks subject to stringent customer due diligence regulations (Old FATF-recommendation 5 / new FATF-recommendation 10)?


alt.Chart(...)